In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from taskdataset import TaskDataset
import matplotlib.pyplot as plt
import os

In [2]:
import sys

path = os.getcwd()
print(path)
os.chdir('..')
sys.path.append(os.getcwd())

from endpoints import sybil_attack, sybil_reset

dataset_t1 = torch.load("task_1_modelstealing/data/ModelStealingPub.pt")


/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam/task_2_sybilattack


In [3]:
os.chdir(path)
print(os.getcwd())
dataset = torch.load("data/SybilAttack.pt")

/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam/task_2_sybilattack


In [4]:
set_t2 = set(dataset.ids)
set_t1 = set(dataset_t1.ids)

intersection = set_t2.intersection(set_t1)

print(intersection)


set()


In [5]:
index = dataset.ids[0]
print(index)
api_ret = sybil_attack(ids=[101031],
                 home_or_defense='home',
                 binary_or_affine='affine')
print(len(api_ret[0]))

101031


TypeError: 'module' object is not callable

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class Linear(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

In [ ]:
l1_penalty = l1_weight * sum([p.abs().sum() for p in net.hidden.parameters()])